In [117]:
import pandas as pd
young_alchemy_data = pd.read_csv('young_alchemy.csv')
old_cols = young_alchemy_data.columns
rename_map = {}
_new_cols = ["title","material_analog","main_effect","side_effect","idnetifier"]
for old_col,new_col in zip(old_cols,_new_cols):
    rename_map[old_col] = new_col

types = ["Магические Металлы","Магические Компоненты","Травы"]
filtered_young_alchemy_data =young_alchemy_data[~young_alchemy_data['Мат. эквивалент'].apply(lambda x: isinstance(x,str) and any(t in x for t in types))]
filtered_young_alchemy_data=filtered_young_alchemy_data.apply(lambda x: x.strip if isinstance(x,str) else x)
filtered_young_alchemy_data.rename(columns=rename_map,inplace=True)


In [73]:
main_effects = []
side_effects = []
curr_main_effect = ""
for _, row in filtered_young_alchemy_data.iterrows():
    if isinstance(row["title"],str) and len(row["title"].strip())>0:
        main_effects.append(curr_main_effect.strip())
        curr_main_effect =""
    if isinstance(row["main_effect"],str) and len(row["main_effect"].strip())>0:
        curr_main_effect = curr_main_effect + row["main_effect"]
    if isinstance(row["side_effect"],str) and len(row["side_effect"].strip())>0:
        side_effects.append(row["side_effect"].strip())
main_effects.append(curr_main_effect.strip())
main_effects = [x for x in main_effects if x not in ["",None] and isinstance(x,str)]
all_effects = main_effects+side_effects
all_effects = list(set(all_effects))
all_effects = [x for x in all_effects if x not in ["",None] and isinstance(x,str)]

In [ ]:
"""Противоядие от смертельных ядов блокирует: 1 смертельный яд в формули или 2 сильных яда в формуле или 3 средних яда в формуле или 4 слабых яда в формуле.
Так же оно может работать по принципу: Смертельное противоядие, блокирует 1 сильный яд и 2 средних яда в 1 формуле например

Бодрость - это физическое состояние, ликвидируется противоположным. 
Галлюцинации - это ментальное состояние можно ликвидировать или прояснением сознания, или например снятием ментального воздействия, так как оно к нему относится.
Снотворное - ликвидируется только бодростью
Ну и побуждение говорить правду - это может ликвидироваться наоборот заставляет врать

"Восстановление функционирования составляющей части личности Разум
Эффект ликвидируется противоположным
«Затмевает ""разум"", хочется прыгать скакать громко кричать И-Го-Го, бить копытом и т.д."

"""

In [213]:
poisons_map ={"Смертельный яд":("яд",-4),
 "Сильный яд":("яд",-3),
 "Средний яд":("яд",-2),
 'Средний Яд (открывает кровотечение)':("яд",-2),
 "Слабый яд":("яд",-1),
 'Смертельный яд для перевёртышей (обладатели таланта "Превращение")':("яд",-4),
 'Способно спасти от смертельных ядов, противоядие':("яд",4),
 'Наносит повреждения магическим созданиям (работает как слабый яд для существ)':("яд",-1),
 'Слабый яд (понижает энергию)':("яд",-1),
 'Сильное Противоядие':("яд",3),
 'Среднее Противоядие':("яд",2),
 'Слабое Противоядие':("яд",1),
 'Противоядие против средних ядов':("яд",2),
 'Ядовито для женщин, при контакте приводик к сну без сноведений (правила магической комы). У мужчин вызывает опьянение и эфорию.':("яд",-1),
 }
sleep_map = {
    'Вводит в летаргический сон':("сон",-4),
    'Эффект снотворного':("сон",-1),
    'Снотворное средство':("сон",-2),
    'Сильное снотворное':("сон",-3),
    'Согревает изнутри, используется для пробуждающих эликсиров (можно использовать такие для снятия стазиса и других эффектов насильного погружения в сны)':("сон",4),
    'Тонизирует (более слабый вариант бодрости)':("сон",2),
    'Бодрость (способность находиться в состоянии высокого жизненного тонуса, избытка сил и желания что-либо делать)':("сон",3),
    'Невозможно уснуть':("сон",4),
    'Выводит из депрессии, повышает общий тонус организма, бодрит':("сон",2),
    
}
depression_map = {
    'Вводит в состояние депрессии':("депрессия",-1),
    'Выводит из депрессии, повышает общий тонус организма, бодрит':("депрессия",1),
    'Лечит сумасшествие, нервные расстройства':("депрессия",1),
}
hallucination_map = {
    'Вызывает Галлюцинации (образы исполненой мечты/грёзы)':("галлюцинации",-1),
    'Галлюцинации, вызывающие злобу, агрессивность, манию пресследования':("галлюцинации",-1),
    'Галлюцинации':("галлюцинации",-1),
    'Лечит сумасшествие, нервные расстройства':("галлюцинации",1),
}
shizofrenia_map = {
    'Путанность мыслей, невозможность их сформулировать':("шизофрения",-1),
    'Фрагментарное ухудшение памяти':("шизофрения",-1),
    'Потеря координации':("шизофрения",-1),
    'Головная боль в височной части':("шизофрения",-1),
    'Головокружение, проблемы с вестибулярным аппаратом':("шизофрения",-1),
    'Вызывает головную боль в затылочной части':("шизофрения",-1),
    'Лечит сумасшествие, нервные расстройства':("шизофрения",1),
    'Сводят с ума':("шизофрения",-1),
}
mental_shield_map = {
    'Ослабляет естественные барьеры (снижает по ступени эликсира показатель барьеров воли, разума и души в равном кол-ве)':("ментальная защита",-1),
    'Улучшает ментальную защиту':("ментальная защита",1),
    'Ослабляет ментальную защиту':("ментальная защита",-1),
    'Повышает ментальную защиту':("ментальная защита",1),
    'Барьер Души снижается до 1 единицы':("ментальная защита",-1),
}
mental_power_map = {
    'Усиливает техники энергии "Разума" (сила эликсира/2 = + к техникам)':("ментальная сила",1),
}
mental_effects_map = {
    'Против ментального воздействия':("ментальные эффекты",1),
    'Разрушает наложенное ментальное воздействие':("ментальные эффекты",1),
}
mental_effects_protection_map = {
    'Защищает от ментального воздействия':("ментальные эффекты защита",1),
}
mental_disease_map = {
    'Лечение магических болезней (ментальных)':("ментальные болезни",1),
}
invisible_detection_map = {
    'Позволяет видеть невидимое и скрытое (работает как плюс "Глаза Существа - Элементаль")':("детектор невидимости",1),
     'Позволяет видеть сквозь красную вуаль (работает как плюс "Глаза Существа - Элементаль")':("детектор невидимости",1),
}
lie_map = {
    'Побуждает говорить только правду':("правда",1),
    'Заставляет говорить только правду':("правда",2),
    'Заставляет непрерывно врать':("правда",-2),
}
attractiveness_map={
    'Повышает привлекательность в глазах представителей противоположного пола, может использоваться в приворотных эликсирах.':("привлекательность",1),
    'Может использоваться в эликсирах дарующих красоту (красная Роза)':("привлекательность",1),
}
attraction_map={
    'Уменьшает влечение к противоположному полу':("влечение",1),
    'Снижает влечение к противоположному полу':("влечение",1),
    'Усиливает влечение к противоположному полу, афродизиак':("влечение",-1),
    'Состояние влюбленности (в первое существо (разумное) противоположного пола, которое увидел)':("влечение",-2),
    'Невосприимчивость к привлекательности противоположного пола (как магической, так и природной)':("влечение",4),
    'Защищает от суккубов и инкубов (любых видов соблазнения, убирает воздействия афродизиаков и приворотов)':("влечение",4),
    'Понижает половое влечение':("влечение",-1),
}
ear_map={
    'Возвращает утерянный слух':("слух",1),
}
eye_map={
    'Возвращает утерянное зрение':("зрение",1),
}
sensation_map={
    'Возвращает телу чувствительность':("чувствительность",1),
}
nightmares_map={'Усиливает страхи, описанный кошмар в вашей вводной для вас теперь реален и вы его видите в любой отражающей поверхности':("кошмары",-1),
          'Спасает от кошмаров':("кошмары",1),
          'Вызывает панику':("кошмары",-1),
          'Повышает храбрость и мужество':("кошмары",1),
          'Бесстрашие и безрассудство':("кошмары",1),
}
fear_map={
}
invisible_map={
    'Обладает свойствами невидимости (красная вуаль)':("невидимость",1),
    'Делает не видимым (по правилам красной вуали)':("невидимость",1),
    'Проявляет невидимое':("невидимость",-1),
}
change_appearance_map={
    'Используется для смены внешности':("изменение внешности",1),
}
increase_potions_map={
    'Усиливает любой эликсир/зелье на выходе. Сила усиления зависит от ступени приготовления зелья/эликсира. Усиление идет +показатель ступени.':("усиление зелий",1),
    'Усиливает готовый эликсир. Сила усиления равна ступени вашего эликсира/зелья (была ступень суперио, усиление идет на +3 единицы и т.д.)':("усиление зелий",+3),
}
sedative_map={
    'Сильное успокоительное':("успокоительное",3),
    'Уравновешенность':("успокоительное",2),
    'Мстительность, агрессия':("успокоительное",-1),
    'Повышает настроение до здоровой веселости':("успокоительное",1),
    'От перепадов Настроения':("успокоительное",1),
    'Повышенная холоднокровность в поступках':("успокоительное",1),
    'Эйфория':("успокоительное",2),
    'Увеличивает Легкомыслие':("успокоительное",-1),
    'СИЛЬНО повышает настроение':("успокоительное",2),
    'Вспыльчивость':("успокоительное",-1),
}
disiease_map={
    '"Морская" болезнь, тошнота':("болезнь",-1),
    'Лихорадка':("болезнь",-1),
    'Снимает воспаление и зуд кожных покровов':("болезнь",1),
    'Способно вылечить средний тип болезней':("болезнь",2),
}
deamon_attract_map={
    'Отпугивает Дэймонов':("дэймоны",1),
}
drunkenness_map={}
all_effects_list=list(poisons_map.items())+list(sleep_map.items())+list(depression_map.items())+list(hallucination_map.items())+list(shizofrenia_map.items())+list(mental_shield_map.items())+list(mental_power_map.items())+list(mental_effects_map.items())+list(mental_effects_protection_map.items())+list(mental_disease_map.items())+list(invisible_detection_map.items())+list(lie_map.items())+list(attractiveness_map.items())+list(attraction_map.items())+list(ear_map.items())+list(eye_map.items())+list(sensation_map.items())+list(fear_map.items())+list(invisible_map.items())+list(nightmares_map.items())+list(change_appearance_map.items())+list(increase_potions_map.items())+list(sedative_map.items())+list(disiease_map.items())+list(deamon_attract_map.items())+list(drunkenness_map.items())
all_effects_list_u=[]
for effect_name,(effect_type,effect_value) in all_effects_list:
    all_effects_list_u.append((effect_name,effect_type,effect_value))
all_effects_df=pd.DataFrame(all_effects_list_u,columns=["effect_name","effect_type","effect_value"])


In [217]:
pd.to_pickle(all_effects_df,"all_effects_df.pkl")

In [216]:
not_mapped = [x for x in all_effects if x.lower() not in [a.lower() for a in all_effects_df["effect_name"]]]
# [x for x in not_mapped if any(a in x.lower() for a in ["сон","сны","сна","бодро","проб"])]
# [x for x in not_mapped if any(a in x.lower() for a in ["лож", "правд","лга","вра"])]
# [x for x in not_mapped if any(a in x.lower() for a in ["храб"])]
# [x for x in not_mapped if any(a in x.lower() for a in ["усп"])]
not_mapped

['В составе эликсира дает магу защиту от различных духов и призраков, защищая от проникновения их в тело.',
 'Дает прозрение (одноразовая сработка таланта "Третий Глаз" на 1 ступени)',
 'Дарует магический щит, работающий только против магии энергии души (стихийная магия, стихийные таланты)',
 'Вызывает "Рыжиковое Заболевание" (после приема зелья/эликсира с данным свойством получить аскольд-карту заболевания у МГ)',
 'Увеличивает Физическое здоровье до показателя силы крови (хиты=сила крови)',
 'Усиливает готовый эликсир (сила на выходе *2) и даёт иммунитет к магии (пока работает этот эликсир полный иммунитет к любым магическим воздествиям)',
 'Вызывает непереносимость солнечного света',
 'Дает неуязвимость к стихийной магии и энергии Души',
 'Вызывает Магическую Лихорадку (получи карту вируса у МГ)',
 'Получение минуса "Любопытная Варвара" (только по средством мутации). При использовании в зельях минус появляется временно, пока работает зелье.',
 'Позволяет понимать язык животных',
 'В